In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import os, sys, re, pickle, wget, math
from intermine.webservice import Service
from IPython.display import Markdown, display
import matplotlib.pyplot as plt
import scipy.stats as stats

# Load data direct from source
class A(object):
    data = {}
    url = {}
    url['expression'] = 'https://ndownloader.figshare.com/files/16757690'
    url['sample_info'] = 'https://ndownloader.figshare.com/files/16757723'
    url['copy_number'] = 'https://ndownloader.figshare.com/files/17857886'
    url['mutations'] = 'https://ndownloader.figshare.com/files/16757702'
    url['achilles_crispr'] = 'https://ndownloader.figshare.com/files/16757666'
    url['d2_rnai'] = 'https://ndownloader.figshare.com/files/13515395'
    url['sensitivity'] = 'https://ndownloader.figshare.com/files/17008628'
    url['mfr'] = 'http://bmbl.sdstate.edu/MFR/data/resource%20data/tr_dv_ts.dataset.zip'
    url['reactome'] = 'https://reactome.org/download/current/NCBI2Reactome.txt'
    summary = '../index.md'
    
# Wipe the slate clean on the analysis summary
os.system("echo '' > "+A.summary)
    
def printmd(string):
    """Prints formatted markdown text"""
    display(Markdown(string))
    
def report(text,silent=False):
    """Print markdown in this notebook and saves markdown-only summary in a file"""
    if not silent:
        printmd(text)
    
    f = open(A.summary,'a')
    f.write(text+'\n')
    
def get_gene_descriptions():
    """Load gene names and descriptions from humanmine (http://www.humanmine.org),
    an integrated database of human genome information.  Use cached data if available"""
    
    archive = 'data/gene_info.p'
    if os.path.exists(archive):
        df = pd.read_pickle(archive)
        return df
    
    service = Service("https://www.humanmine.org/humanmine/service")
    query = service.new_query("Gene")
    cols = ["primaryIdentifier", "symbol", "briefDescription", "description","proteins.uniprotAccession"]
    query.add_view(*cols)
    query.add_constraint("organism.taxonId", "=", "9606", code = "A")    
    df_rows = []

    for row in query.rows():
        df_rows.append(
            [row["primaryIdentifier"], 
             row["symbol"], 
             row["briefDescription"], 
             row["description"],
             row["proteins.uniprotAccession"]
            ])

    df = pd.DataFrame(data=df_rows,columns=cols)
    df.to_pickle(archive)
    return df
    
def get_data(key):
    """Load input data.
    Arguments: key for the data source (eg: expression, sample_info...)
    1) If the data is cached on the filesystem, load and return the dataframe
    2) Otherwise, load the data from the source URL, cache, return the dataframe
    """
    
    data_cache = 'data/'+key+'.p'
    if os.path.exists(data_cache):
        df = pd.read_pickle(data_cache)
    else:
        print("Downloading",key,"from source")
        df = pd.read_csv(A.url[key],low_memory=False,index_col=0)
        df.to_pickle(data_cache)
        
    A.data[key] = df    
    return df

def ncbi_gene_ids(genes):
    """Converts gene name "symbol (ncbi_id)"
    to integer NCBI ID.  Also catch missing associations
    for NCBI gene ID/symbol
    """
    ncbi_cols = []
    for g in genes:
        match = re.search('(\S+)\s+\((\d+)\)',g)
        if match:
            s = match.group(1)
            g = match.group(2)
            if not ncbi2symbol.get(int(g)):
                ncbi2symbol[int(g)] = s
                symbol2ncbi[s] = int(g)
        else:
            print("No match",g)
        ncbi_cols.append(int(g))
        

    return ncbi_cols

def get_pathway_info():
    # Map NCBI IDs to reactome pathways
    # File downloaded from https://reactome.org/download/current/NCBI2Reactome.txt
    archive = 'data/pathway_info.p'
    
    if os.path.exists(archive):
        pathway_info = pickle.load(open(archive,'rb'))
        return pathway_info
    
    pathway_info = {}
    if not os.path.exists('data/NCBI2Reactome.txt'):
        wget.download(A.url['reactome'],out='data/NCBI2Reactome.txt')
    with open('data/NCBI2Reactome.txt') as n2r:
        for line in n2r.readlines():
            ncbi, pathway_id, url, pathway_name, type, species = line.strip().split('\t')
            # only human pathways
            if species != 'Homo sapiens':
                continue
            # only curated pathways
            if type == 'IEA':
                continue
            pathway_info[ncbi] = [pathway_id, url, pathway_name]

    pickle.dump(pathway_info,open(archive,'wb'))
    return pathway_info


In [2]:
report('''
# Sample analysis of Cancer Dependency Map (DepMap) data 
## Request
* Identify the most frequent genetic alterations (could be mutations or copy number variations) in the cancer cell lines
* Match them with the best genetic dependencies that could be used for drug development for the cancers that carry those mutations
* Take into account the lineage of cancer cell lines (certain mutations/CNVs may be restricted to a specific lineage)

## Resources
### DepMap (https://depmap.org/portal) Data 
* Cell line metadata
* Expression (RNASeq)
* Copy number variation
* Mutations
* Genetic dependency
  * Crispr (Achilles)
  * RNAi (DEMETER2)
  
<b>Citations:</b> 
* Jordan G. Bryan, John M. Krill-Burger, Thomas M. Green, Francisca Vazquez, Jesse S. Boehm, Todd R. Golub, William C. Hahn, David E. Root, Aviad Tsherniak. (2018). Improved estimation of cancer dependencies from large-scale RNAi screens using model-based normalization and data integration. Nature Communications 9, 1. https://doi.org/10.1038/s41467-018-06916-5</font>
* DepMap, Broad (2019): DepMap 19Q3 Public. figshare. Dataset doi:10.6084/m9.figshare.9201770.v1.
* Robin M. Meyers, Jordan G. Bryan, James M. McFarland, Barbara A. Weir, ... David E. Root, William C. Hahn, Aviad Tsherniak. Computational correction of copy number effect improves specificity of CRISPR-Cas9 essentiality screens in cancer cells. Nature Genetics 2017 October 49:1779–1784. doi:10.1038/ng.3984


### NCBI (via https://humanmine.org)
* Entrez gene IDs mapped to symbol, name, descrion, uniprot

### Reactome
* Entrez gene IDs mapped to Reactome pathways

### MFR (http://bmbl.sdstate.edu/MFR)
* A Machine Learning Model for measuring relatedness between a pair of genes

### Jupyter (https://jupyter.org/)
* Python programming framework for analysis prototyping and reporting

### GitHub (https://github.com/)
* Revision control for Python code
* Reporting mechanism for analysi summary and details
''')


# Sample analysis of Cancer Dependency Map (DepMap) data 
## Request
* Identify the most frequent genetic alterations (could be mutations or copy number variations) in the cancer cell lines
* Match them with the best genetic dependencies that could be used for drug development for the cancers that carry those mutations
* Take into account the lineage of cancer cell lines (certain mutations/CNVs may be restricted to a specific lineage)

## Resources
### DepMap (https://depmap.org/portal) Data 
* Cell line metadata
* Expression (RNASeq)
* Copy number variation
* Mutations
* Genetic dependency
  * Crispr (Achilles)
  * RNAi (DEMETER2)
  
<b>Citations:</b> 
* Jordan G. Bryan, John M. Krill-Burger, Thomas M. Green, Francisca Vazquez, Jesse S. Boehm, Todd R. Golub, William C. Hahn, David E. Root, Aviad Tsherniak. (2018). Improved estimation of cancer dependencies from large-scale RNAi screens using model-based normalization and data integration. Nature Communications 9, 1. https://doi.org/10.1038/s41467-018-06916-5</font>
* DepMap, Broad (2019): DepMap 19Q3 Public. figshare. Dataset doi:10.6084/m9.figshare.9201770.v1.
* Robin M. Meyers, Jordan G. Bryan, James M. McFarland, Barbara A. Weir, ... David E. Root, William C. Hahn, Aviad Tsherniak. Computational correction of copy number effect improves specificity of CRISPR-Cas9 essentiality screens in cancer cells. Nature Genetics 2017 October 49:1779–1784. doi:10.1038/ng.3984


### NCBI (via https://humanmine.org)
* Entrez gene IDs mapped to symbol, name, descrion, uniprot

### Reactome
* Entrez gene IDs mapped to Reactome pathways

### MFR (http://bmbl.sdstate.edu/MFR)
* A Machine Learning Model for measuring relatedness between a pair of genes

### Jupyter (https://jupyter.org/)
* Python programming framework for analysis prototyping and reporting

### GitHub (https://github.com/)
* Revision control for Python code
* Reporting mechanism for analysi summary and details


In [3]:
report('''
## Get gene descriptions, etc.
Use data from humanmine (http://www.humanmine.org/) to map NCBI gene IDs to name, summary, symbol, uniprot
''')


## Get gene descriptions, etc.
Use data from humanmine (http://www.humanmine.org/) to map NCBI gene IDs to name, summary, symbol, uniprot


In [4]:
gd = get_gene_descriptions()
sample = gd.head(2).to_html(index=False)
sample = re.sub('The protein.+</','The protein encoded by this gene...</',sample)
sample = re.sub('This gene encodes.+</','This gene encodes...</',sample)
report('sample gene info:\n'+sample,silent=True)
ncbi2name    = {}
ncbi2symbol  = {}
symbol2ncbi  = {}
ncbi2desc    = {}
ncbi2uniprot = {}
uniprot2ncbi = {}

for i, r in gd.iterrows():
    ncbi, symbol, name, description, uniprot = list(r)
    ncbi = int(ncbi)
    ncbi2name[ncbi] = name
    ncbi2symbol[ncbi] = symbol
    ncbi2desc[ncbi] = description
    
    # ncbi <-> uniprot can be 1:many
    if ncbi2uniprot.get(ncbi) is None:
        ncbi2uniprot[ncbi] = set()
    ncbi2uniprot[ncbi].add(uniprot)
    uniprot2ncbi[uniprot] = ncbi
    
print("Done mapping gene info")

Done mapping gene info


In [5]:
report('''
## Aggregate CCLE cell lines by lineage
<b>DepMap source file:</b> sample_info.csv
* Group all cell lines (CCLE cell line IDs) by main (parent) lineage
* If a lineage has > 1 defined sublineages, also aggregate cell lines by sublineage (eg: leukemia -> AML)
''')


## Aggregate CCLE cell lines by lineage
<b>DepMap source file:</b> sample_info.csv
* Group all cell lines (CCLE cell line IDs) by main (parent) lineage
* If a lineage has > 1 defined sublineages, also aggregate cell lines by sublineage (eg: leukemia -> AML)


In [6]:
sample_info = get_data('sample_info')
lineages = set(sample_info.lineage.dropna())

lineage2cell    = {}
sublineage2cell = {}
cell2lineage    = {}
cell2sublineage = {}
has_sub = set()
is_sub = set()

for l in lineages:
    ldf = sample_info[sample_info.lineage == l]
    subtypes = set(ldf.lineage_subtype.dropna())
    
    # cell lines for sublineage
    if len(subtypes) > 1:
        has_sub.add(l)
        for sub in subtypes:
            if l in sub:
                lname = sub
            else:
                lname = l + '_' + sub

            is_sub.add(lname)
            sub_df = ldf[ldf.lineage_subtype == sub]

            # map sublineage to cell lines
            sublineage2cell[lname] = list(sub_df.index)
    
            # map cell lines to sub-lineage
            for cell in sublineage2cell[lname]:
                cell2sublineage[cell] = lname
    else:
        sublineage2cell[l] = list(ldf.index)
        for cell in sublineage2cell[l]:
            cell2sublineage[cell] = l
                
    # all cell lines for lineage
    lineage2cell[l] = list(ldf.index)
    
    # parent lineage of each cell line
    for cell in lineage2cell[l]:
        cell2lineage[cell] = l
    

report('<pre>')
report("Number of cell lines: "+str(len(cell2lineage)))
report("Number of lineages: "+str(len(lineage2cell)))
report("Number of lineages with sub-lineages: "+str(len(has_sub)))
report("Number of sub-lineages "+str(len(is_sub)))
report('</pre>')

<pre>

Number of cell lines: 1429

Number of lineages: 33

Number of lineages with sub-lineages: 16

Number of sub-lineages 61

</pre>

In [7]:
report('''
## Expression Data (RNASeq)
RNAseq TPM gene expression data for just protein coding genes using RSEM. Log2 transformed, using a pseudo-count of 1.

<b>DepMap source file:</b> CCLE_expression.csv
* Transpose columns (gene names) and rows (CCLE cell line IDs)
* Translate gene names to NCBI gene IDs
* Apply BOOLEAN mask to accept log2 tpm > 0 as positive for expression
''')


## Expression Data (RNASeq)
RNAseq TPM gene expression data for just protein coding genes using RSEM. Log2 transformed, using a pseudo-count of 1.

<b>DepMap source file:</b> CCLE_expression.csv
* Transpose columns (gene names) and rows (CCLE cell line IDs)
* Translate gene names to NCBI gene IDs
* Apply BOOLEAN mask to accept log2 tpm > 0 as positive for expression


In [8]:
exp = get_data('expression').T
exp.index = ncbi_gene_ids(list(exp.index))
report('Sample expression data: '+exp.iloc[:3,:10].to_html())
exp_masked = exp > 0
report('<br>Sample masked expression: '+exp_masked.iloc[:3,:10].to_html())

Sample expression data: <table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>ACH-001097</th>
      <th>ACH-001485</th>
      <th>ACH-001396</th>
      <th>ACH-000534</th>
      <th>ACH-000742</th>
      <th>ACH-001818</th>
      <th>ACH-000545</th>
      <th>ACH-000836</th>
      <th>ACH-001959</th>
      <th>ACH-000470</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>7105</th>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>2.883621</td>
      <td>0.839960</td>
      <td>3.722466</td>
      <td>4.032982</td>
      <td>4.251719</td>
      <td>4.632268</td>
      <td>3.321928</td>
      <td>3.681449</td>
    </tr>
    <tr>
      <th>64102</th>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>0.000000</td>
    </tr>
    <tr>
      <th>8813</th>
      <td>4.667324</td>
      <td>5.755689</td>
      <td>4.471838</td>
      <td>5.376082</td>
      <td>6.029674</td>
      <td>5.933100</td>
      <td>5.651052</td>
      <td>6.704180</td>
      <td>7.357288</td>
      <td>7.294896</td>
    </tr>
  </tbody>
</table>

<br>Sample masked expression: <table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>ACH-001097</th>
      <th>ACH-001485</th>
      <th>ACH-001396</th>
      <th>ACH-000534</th>
      <th>ACH-000742</th>
      <th>ACH-001818</th>
      <th>ACH-000545</th>
      <th>ACH-000836</th>
      <th>ACH-001959</th>
      <th>ACH-000470</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>7105</th>
      <td>False</td>
      <td>False</td>
      <td>True</td>
      <td>True</td>
      <td>True</td>
      <td>True</td>
      <td>True</td>
      <td>True</td>
      <td>True</td>
      <td>True</td>
    </tr>
    <tr>
      <th>64102</th>
      <td>False</td>
      <td>False</td>
      <td>False</td>
      <td>False</td>
      <td>False</td>
      <td>False</td>
      <td>False</td>
      <td>False</td>
      <td>False</td>
      <td>False</td>
    </tr>
    <tr>
      <th>8813</th>
      <td>True</td>
      <td>True</td>
      <td>True</td>
      <td>True</td>
      <td>True</td>
      <td>True</td>
      <td>True</td>
      <td>True</td>
      <td>True</td>
      <td>True</td>
    </tr>
  </tbody>
</table>

In [9]:
report('''
## Mutation Data 
<b>DepMap source file:</b> CCLE_mutations.csv
* Keep track of TCGA and COSMIC hotspot genes by lineage
* Just using hotspot genes for now but track deleterious mutations by lineage for future reference
''')


## Mutation Data 
<b>DepMap source file:</b> CCLE_mutations.csv
* Keep track of TCGA and COSMIC hotspot genes by lineage
* Just using hotspot genes for now but track deleterious mutations by lineage for future reference


In [10]:
mutations = get_data('mutations')

In [11]:
# Filter hotspot genes
hotspots = []
hotspots.append(mutations[mutations.isTCGAhotspot])
hotspots.append(mutations[mutations.isCOSMIChotspot])
hotspots = pd.concat(hotspots).drop_duplicates()

hotspot_genes = set(hotspots.Entrez_Gene_Id)

lineage_hotspots = {}
sublineage_hotspots = {}

for g in hotspot_genes:
    lineage_hotspots[g] = {}
    sublineage_hotspots[g] = {}
    
    gdf = hotspots[hotspots.Entrez_Gene_Id == g]
    
    for i, r in gdf.iterrows():
        cell = r.DepMap_ID
        if cell2lineage.get(cell) is not None:
            lineage = cell2lineage[cell]
            if lineage_hotspots[g].get(lineage) is None:
                lineage_hotspots[g][lineage] = 0
            lineage_hotspots[g][lineage] += 1
            
        if cell2sublineage.get(cell) is not None:
            sublineage = cell2sublineage[cell]
            if sublineage_hotspots[g].get(sublineage) is None:
                sublineage_hotspots[g][sublineage] = 0
            sublineage_hotspots[g][sublineage] += 1
            
print("Done mapping hotspots")
report(str(len(hotspot_genes))+' TCGA or COSMIC hotspot genes')

Done mapping hotspots


8704 TCGA or COSMIC hotspot genes

In [12]:
# Filter deleterious mutations
damaging = mutations[mutations.Variant_annotation.isin(['damaging','non-conserving'])]
damaging = damaging[damaging.isDeleterious == True]
genes = set(hotspots.Entrez_Gene_Id)


genes = set(damaging.Entrez_Gene_Id)

lineage_mutations = {}
sublineage_mutations = {}

for g in genes:
    lineage_mutations[g] = {}
    sublineage_mutations[g] = {}
    
    gdf = damaging[damaging.Entrez_Gene_Id == g]
    
    for i, r in gdf.iterrows():
        cell = r.DepMap_ID
        if cell2lineage.get(cell) is not None:
            lineage = cell2lineage[cell]
            if lineage_mutations[g].get(lineage) is None:
                lineage_mutations[g][lineage] = 0
            lineage_mutations[g][lineage] += 1
            
        if cell2sublineage.get(cell) is not None:
            sublineage = cell2sublineage[cell]
            if sublineage_mutations[g].get(sublineage) is None:
                sublineage_mutations[g][sublineage] = 0
            sublineage_mutations[g][sublineage] += 1
            
print("Done mapping deleterious mutations")

Done mapping deleterious mutations


In [21]:
report('''
## Copy number data
Gene level copy number data, log2 transformed with a pseudo count of 1. This is generated by mapping genes onto the segment level calls.

<b>Depmap source file:</b> CCLE_gene_cn_v2.csv

* Transpose columns (gene names) and rows (CCLE cell line IDs)
* Translate gene names to NCBI gene IDs
''')



## Copy number data
Gene level copy number data, log2 transformed with a pseudo count of 1. This is generated by mapping genes onto the segment level calls.

<b>Depmap source file:</b> CCLE_gene_cn_v2.csv

* Transpose columns (gene names) and rows (CCLE cell line IDs)
* Translate gene names to NCBI gene IDs


In [178]:
cn = get_data('copy_number')
cn.columns = ncbi_gene_ids(list(cn.columns))

# cn.columns = ncbi_gene_ids(list(cn.columns))
# print(cn.shape)
# keep_genes = []
# for g in cn.columns:
#     if g in hotspot_genes:
#         keep_genes.append(g)
# cn = cn[keep_genes]
# print(cn.shape)
cn = cn.T
report('Sample copy number data: '+cn.iloc[:3,:10].to_html())
report('Samlpe copy number data descritive stats: '+cn.iloc[:,:10].describe().to_html())

Sample copy number data: <table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>ACH-001793</th>
      <th>ACH-002176</th>
      <th>ACH-000652</th>
      <th>ACH-001295</th>
      <th>ACH-000798</th>
      <th>ACH-001399</th>
      <th>ACH-000111</th>
      <th>ACH-002358</th>
      <th>ACH-000367</th>
      <th>ACH-000584</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <td>1.086422</td>
      <td>1.526188</td>
      <td>0.776609</td>
      <td>0.964857</td>
      <td>0.986651</td>
      <td>1.097441</td>
      <td>1.577719</td>
      <td>0.990491</td>
      <td>0.972714</td>
      <td>1.232228</td>
    </tr>
    <tr>
      <th>10</th>
      <td>1.103066</td>
      <td>1.044267</td>
      <td>0.520576</td>
      <td>1.001427</td>
      <td>0.974178</td>
      <td>0.775405</td>
      <td>1.209647</td>
      <td>1.001602</td>
      <td>0.564791</td>
      <td>0.379099</td>
    </tr>
    <tr>
      <th>100</th>
      <td>0.910926</td>
      <td>1.089161</td>
      <td>1.555780</td>
      <td>1.010086</td>
      <td>0.987801</td>
      <td>1.877675</td>
      <td>0.826138</td>
      <td>0.999580</td>
      <td>1.128533</td>
      <td>1.254932</td>
    </tr>
  </tbody>
</table>

Samlpe copy number data descritive stats: <table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>ACH-001793</th>
      <th>ACH-002176</th>
      <th>ACH-000652</th>
      <th>ACH-001295</th>
      <th>ACH-000798</th>
      <th>ACH-001399</th>
      <th>ACH-000111</th>
      <th>ACH-002358</th>
      <th>ACH-000367</th>
      <th>ACH-000584</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>count</th>
      <td>27639.000000</td>
      <td>2.763900e+04</td>
      <td>2.763900e+04</td>
      <td>2.763900e+04</td>
      <td>27639.000000</td>
      <td>27639.000000</td>
      <td>2.763900e+04</td>
      <td>27639.000000</td>
      <td>2.763900e+04</td>
      <td>27639.000000</td>
    </tr>
    <tr>
      <th>mean</th>
      <td>1.098416</td>
      <td>1.005611e+00</td>
      <td>9.739989e-01</td>
      <td>1.061859e+00</td>
      <td>1.019288</td>
      <td>1.057222</td>
      <td>1.127191e+00</td>
      <td>0.980485</td>
      <td>9.532559e-01</td>
      <td>1.057527</td>
    </tr>
    <tr>
      <th>std</th>
      <td>1.119507</td>
      <td>3.482645e-01</td>
      <td>2.416796e-01</td>
      <td>6.685549e-01</td>
      <td>0.123101</td>
      <td>0.306522</td>
      <td>3.919352e-01</td>
      <td>0.092264</td>
      <td>2.693422e-01</td>
      <td>0.366487</td>
    </tr>
    <tr>
      <th>min</th>
      <td>0.171960</td>
      <td>7.998209e-10</td>
      <td>8.220704e-10</td>
      <td>7.984062e-10</td>
      <td>0.000036</td>
      <td>0.000020</td>
      <td>1.102449e-09</td>
      <td>0.000021</td>
      <td>8.282854e-10</td>
      <td>0.000009</td>
    </tr>
    <tr>
      <th>25%</th>
      <td>0.889299</td>
      <td>7.318536e-01</td>
      <td>7.900758e-01</td>
      <td>9.845272e-01</td>
      <td>0.982870</td>
      <td>0.812673</td>
      <td>8.295737e-01</td>
      <td>0.993549</td>
      <td>6.015790e-01</td>
      <td>0.700131</td>
    </tr>
    <tr>
      <th>50%</th>
      <td>1.053484</td>
      <td>1.045336e+00</td>
      <td>1.027553e+00</td>
      <td>9.942944e-01</td>
      <td>0.986097</td>
      <td>1.003401</td>
      <td>9.373404e-01</td>
      <td>0.999580</td>
      <td>1.056217e+00</td>
      <td>0.990265</td>
    </tr>
    <tr>
      <th>75%</th>
      <td>1.113955</td>
      <td>1.111985e+00</td>
      <td>1.068509e+00</td>
      <td>1.002200e+00</td>
      <td>0.989550</td>
      <td>1.120454</td>
      <td>1.321072e+00</td>
      <td>1.002886</td>
      <td>1.131187e+00</td>
      <td>1.262886</td>
    </tr>
    <tr>
      <th>max</th>
      <td>35.734800</td>
      <td>4.952679e+00</td>
      <td>4.804283e+00</td>
      <td>3.963217e+01</td>
      <td>1.462469</td>
      <td>3.217691</td>
      <td>3.229879e+00</td>
      <td>1.006658</td>
      <td>1.729972e+00</td>
      <td>3.354475</td>
    </tr>
  </tbody>
</table>

In [98]:
report('''
## Search for high copy number genes in each of the lineages
* Seleting genes with copy number > 2 in <b><i>all</i></b> cell lines is a lineage is a bit too stringent
* Retain genes that have copy number >= 2 in 80% or more cell lines in a lineage
* Extract the data for any cell lines with high copy number genes to use as plotting baseline
''')


## Search for high copy number genes in each of the lineages
* Seleting genes with copy number > 2 in <b><i>all</i></b> cell lines is a lineage is a bit too stringent
* Retain genes that have copy number >= 2 in 80% or more cell lines in a lineage
* Extract the data for any cell lines with high copy number genes to use as plotting baseline


In [179]:
# for each cell line, get all of the genes with copy number > 2
keep_common_genes = []
keep_common_cells = []
keep_lineage = set()
save_rows = []

report('<pre>')
for sublin in sublineage2cell:
    cells = sublineage2cell[sublin]
    #print(sublin,len(cells))
    
    best = {}
    for cell in cells:
        try:
            cdf = cn[cell]
            cdf = cdf[cdf >= 2]
            best[cell] = set(cdf.index)
            for g in cdf.keys():
                row = [cell,sublin,ncbi2symbol[g],g,cdf[g]]
                save_rows.append(row)
        except Exception as e:
            #print('Error',e)
            continue
            
    
    best_genes = list(best.values())
    set1 = best_genes.pop()
    all_genes = set1.union(*best_genes)

    total = len(best_genes) + 1
    common_genes = []
    for g in all_genes:
        gcount = 0
        for gset in [set1,*best_genes]:
            if g in gset:
                gcount += 1
        if gcount / total >= 0.8:
            common_genes.append(g)
        
        
    keep_common_genes.extend(common_genes)
    common_symbols = [ncbi2symbol.get(g) for g in common_genes]
    if len(common_genes) > 0:
        keep_lineage.add(sublin)
        keep_common_cells.extend(cells)
        report(sublin+'; '+str(len(cells))+' cell lines; '+str(len(common_genes))+' high copy number genes')
report('</pre>')
        

keep_common_genes = set(keep_common_genes)
keep_common_cells = set(keep_common_cells)

<pre>

soft_tissue_sarcoma_undifferentiated; 2 cell lines; 164 high copy number genes

soft_tissue_fibrosarcoma; 1 cell lines; 2 high copy number genes

soft_tissue_epitheliod_sarcoma; 2 cell lines; 2 high copy number genes

soft_tissue_liposarcoma; 5 cell lines; 44 high copy number genes

...

skin_epidermoid_carcinoma; 1 cell lines; 234 high copy number genes

lymphoblastic_lymphoma; 1 cell lines; 311 high copy number genes

lung_immortalized; 1 cell lines; 6 high copy number genes

upper_aerodigestive_buccal_mucosa; 1 cell lines; 592 high copy number genes

</pre>

In [181]:
keep_genes = set()
for g in ac_genes:
    if g in keep_common_genes:
        keep_genes.add(g)

keep_cells = set()
for c in ac_cells:
    if c in keep_common_cells:
        keep_cells.add(c)
        
ac_s = achilles[keep_cells]
ac_s = ac_s.T[keep_genes].T
cn_s = cn[keep_cells]
cn_s = cn_s.T[keep_genes].T

ac_data = ac_s.to_numpy().flatten()
cn_data = cn_s.to_numpy().flatten()

# remove NaN and Inf
keep_ac = []
keep_cn = []
for i, num1 in enumerate(ac_data):
    num2 = cn_data[i]
    if not np.isfinite(num1) and np.isfinite(num2):
        continue
    keep_ac.append(num1)
    keep_cn.append(num2)
    
ac_data = keep_ac
cn_data = keep_cn  
cn_orig = cn_data
ac_orig = ac_data

In [28]:
report('''
## DEMETER2 RNAi gene dependency data
Cancer cell line genetic dependencies estimated using the DEMETER2 model. DEMETER2 is applied to three large-scale RNAi screening datasets: the Broad Institute Project Achilles, Novartis Project DRIVE, and the Marcotte et al. breast cell line dataset. The model is also applied to generate a combined dataset of gene dependencies covering a total of 712 unique cancer cell lines.

<b>DepMap source file:</b> D2_combined_gene_dep_scores.csv 

* Data source uses CCLE names rather than DepMap cell line IDS
* Translate the cell line names to IDS for consistency with other data sources
* Also deal with rows in the table with multiple gene names (eg 'GTF2IP4&GTF2IP1 (100093631&2970)')
''')


## DEMETER2 RNAi gene dependency data
Cancer cell line genetic dependencies estimated using the DEMETER2 model. DEMETER2 is applied to three large-scale RNAi screening datasets: the Broad Institute Project Achilles, Novartis Project DRIVE, and the Marcotte et al. breast cell line dataset. The model is also applied to generate a combined dataset of gene dependencies covering a total of 712 unique cancer cell lines.

<b>DepMap source file:</b> D2_combined_gene_dep_scores.csv 

* Data source uses CCLE names rather than DepMap cell line IDS
* Translate the cell line names to IDS for consistency with other data sources
* Also deal with rows in the table with multiple gene names (eg 'GTF2IP4&GTF2IP1 (100093631&2970)')


In [157]:
# Ingest data
d2 = get_data('d2_rnai')
print("Initial number of rows in dataframe:",d2.shape[0])
# split rows with multuple genes
cols = ['gene'] + list(d2.columns)
rows = []

print("Splitting multigene rows...")
for i, r in d2.iterrows():
    if '&' in i:
        symbols, ncbi = i.split(' ')
        symbols = symbols.split('&')
        ncbi = re.sub('\(|\)','',ncbi)
        ncbi = ncbi.split('&')
        assert len(symbols) == len(ncbi), "Length mismatch"
        for s, symbol in enumerate(symbols):
            gid = ncbi[s]
            row = [symbol+' ('+gid+')'] + list(r)
            rows.append(row)
    else:
        rows.append([i]+list(r))
d2 = pd.DataFrame(data=rows,columns=cols).set_index('gene')
print("Final number of rows in dataframe:",d2.shape[0])

Initial number of rows in dataframe: 17309
Splitting multigene rows...
Final number of rows in dataframe: 17731


In [158]:
# Map cell line names to IDs
sample_info = get_data('sample_info')
ccle_name2id = {}
for i, r in sample_info.iterrows():
    ccle_name2id[r['CCLE Name']] = i 

cell_line_names = list(d2.columns)

# Rename columns to use CCLE IDs and rows to use NCBI gene IDs
if isinstance(list(d2.index)[0],str):
    d2.columns = [ccle_name2id.get(i) or i for i in cell_line_names]
    d2.index = ncbi_gene_ids(list(d2.index))
print(d2.shape[0],"genes")
print(d2.shape[1],"cell lines")
d2.head()

17731 genes
712 cell lines


,ACH-001270,ACH-001000,ACH-001001,ACH-002319,ACH-001827,ACH-000956,ACH-000948,ACH-002320,ACH-000070,ACH-000411,...,ACH-000899,ACH-000765,ACH-000534,ACH-000762,ACH-000630,ACH-000570,ACH-001249,ACH-000097,ACH-000828,ACH-002331
1,NaN,NaN,0.146042,-0.190388,0.907063,-0.019331,-0.016734,0.091580,0.035023,-0.122046,...,-0.088267,0.002171,NaN,0.120294,0.012540,0.111530,NaN,-0.079313,-0.141559,0.214268
10,NaN,NaN,0.102854,0.384106,0.403192,0.001925,-0.153933,-0.317969,0.012341,-0.205077,...,-0.003747,-0.321445,NaN,-0.003256,-0.220472,0.073460,NaN,-0.130921,0.127358,-0.405974
100,NaN,NaN,0.168839,-0.120700,0.004394,-0.188700,-0.060818,-0.755058,0.129770,0.076273,...,-0.014085,0.039679,NaN,0.076521,0.106995,0.227977,NaN,-0.134479,0.083506,-0.404291
1000,-0.194962,-0.028171,0.063047,-0.237251,-0.017059,-0.103047,-0.049460,0.130107,0.146864,-0.126198,...,-0.073435,-0.140041,-0.154436,-0.040308,-0.078707,0.000769,-0.139126,0.047022,-0.097644,-0.062622
10000,-0.256108,0.100751,-0.008077,0.060267,-0.094749,-0.066591,0.166029,0.149969,-0.053022,0.092426,...,0.028714,-0.054628,0.450581,0.002932,0.129679,-0.072564,0.017161,0.123615,0.046846,0.125711


In [32]:
report('''
## Achilles Crispr gene dependency data
CERES data with principle components strongly related to known batch effects removed, then shifted and scaled per cell line so the median nonessential KO effect is 0 and the median essential KO effect is -1.

<b>DepMap source file:</b> Achilles_gene_effect.csv 

* Translate gene names (column labels) to NCBI IDS
* Transpose rows and columns so each cell line is a column label with vertivally stacked gene data
''')


## Achilles Crispr gene dependency data
CERES data with principle components strongly related to known batch effects removed, then shifted and scaled per cell line so the median nonessential KO effect is 0 and the median essential KO effect is -1.

<b>DepMap source file:</b> Achilles_gene_effect.csv 

* Translate gene names (column labels) to NCBI IDS
* Transpose rows and columns so each cell line is a column label with vertivally stacked gene data


In [159]:
achilles = get_data('achilles_crispr').T
genes = list(achilles.index)
achilles.index = ncbi_gene_ids(genes)
print(achilles.shape[0],"genes")
print(achilles.shape[1],"cell lines")
report('Sample Achilles data:\n'+achilles.iloc[:2,:10].to_html(),True)
achilles.head(2)

18333 genes
625 cell lines


,ACH-000004,ACH-000005,ACH-000007,ACH-000009,ACH-000011,ACH-000012,ACH-000013,ACH-000014,ACH-000015,ACH-000017,...,ACH-001736,ACH-001737,ACH-001740,ACH-001745,ACH-001750,ACH-001765,ACH-001814,ACH-001838,ACH-001956,ACH-001957
1,0.168684,-0.068759,0.053893,0.059874,0.277165,0.008073,0.062131,0.143078,-0.090890,0.178427,...,0.154567,-0.050307,0.005125,0.208843,0.044674,0.136364,0.216507,-0.086149,-0.076893,0.055750
29974,0.089128,0.218792,0.081444,-0.011153,0.085354,0.167177,0.038687,-0.035837,0.007894,0.106952,...,0.019334,0.189813,0.349099,0.153637,0.126563,0.021261,-0.172366,0.082798,0.109464,0.004545


In [145]:
report('''
### How many cell lines and genes are shared between D2 (RNAi) and Achilles (Crispr) gene dependency data sets?
''')


### How many cell lines and genes are shared between D2 (RNAi) and Achilles (Crispr) gene dependency data sets?


In [160]:
d2_cells = set(d2.columns)
d2_genes = set(d2.index)
ac_cells = set(achilles.columns)
ac_genes = set(achilles.index)
shared_cells = d2_cells.intersection(ac_cells)
shared_genes = d2_genes.intersection(ac_genes)
report('<pre>')
report(str(len(shared_cells))+" cell lines are shared")
report(str(len(shared_genes))+" genes are shared")
report('</pre>')

<pre>

423 cell lines are shared

16052 genes are shared

</pre>

### How do RNAi and Crispr screen compare for gene dependency score?

In [169]:
# filter shared genes to just include our high copy number genes
my_d2 = d2.copy()
keep_genes = set()
for g in shared_genes:
    if g in keep_common_genes:
        keep_genes.add(g)

keep_cells = set()
for c in shared_cells:
    if c in keep_common_cells:
        keep_cells.add(c)
        
d2_s = my_d2[keep_cells]
d2_s = d2_s.T[keep_genes].T
ac_s = achilles[keep_cells]
ac_s = ac_s.T[keep_genes].T


d2_data = d2_s.to_numpy().flatten()
ac_data = ac_s.to_numpy().flatten()

# remove NaN and Inf
keep_d2 = []
keep_ac = []
for i, num1 in enumerate(d2_data):
    num2 = ac_data[i]
    if not np.isfinite(num1) and np.isfinite(num2):
        continue
    keep_d2.append(num1)
    keep_ac.append(num2)
d2_data = keep_d2
ac_data = keep_ac    

In [182]:
# save the data
keep_saved_rows = []
for cn in save_rows:
    cell, lineage, symbol, gene, copynum = cn
    try:
        adata = ac_s.loc[gene]
        adata = adata[cell]
    except:
        continue
    try:
        ddata = d2_s.loc[gene]
        ddata = ddata[cell]
    except:
        ddata = np.nan
    
    # only keep rows with negative gene deendency
    if adata > 0:
        continue

    is_hotspot = gene in hotspot_genes
    
    try:
        exp_data = exp.loc[gene]
        exp_data = exp_data[cell]
        is_expressed = exp_data > 0
    except:
        is_expressed = False
    
    
    keep_saved_rows.append(cn+[adata,ddata,is_hotspot,is_expressed])
    


In [185]:
cols = ['DepMap_ID','sublineage','HUGO_symbol','Entrez_ID','copy_number',
        'achilles_gene_dep','rnai_gene_dep','hotspot','rnaseq_expressed']
candidates = pd.DataFrame(data=keep_saved_rows,columns=cols)

In [153]:
%matplotlib inline
plt.scatter(d2_data,ac_data)
slope, intercept, r_value, p_value, std_err = stats.linregress(d2_data,ac_data)
plt.xlabel('Crispr (Achilles)')
plt.ylabel('RNAi (DEMETER 2)')
plt.title('Gene dependency')
plt.text(-2.6,2.1,'slope='+'%.3f'%slope)
plt.text(-2.6,1.8,'r-squared ='+'%.3f' %r_value)
plt.text(-2.6,1.5,'p-value ='+'%.3f'%p_value)
plt.savefig('plots/gene_dependency.png',dpi=100)
plt.close()

report('<img src="plots/gene_dependency.png" style="float:left">')
report('* significant p-value means reject H0 that slope == 0')
report('* We will use the Achilles (Crispr) gene dependency score and check for positive agreement with RNAi later')

<img src="plots/gene_dependency.png" style="float:left">

* significant p-value means reject H0 that slope == 0

* We will use the Achilles (Crispr) gene dependency score and check for positive agreement with RNAi later

In [89]:
report('''
### Sanity checking with ERBB2 (2064)
Evaluating breast cancer lineages where at least one cell line had copy number > 2:
* Is ERB2B in the hotspot gene set?
* We expect ERBB2 to have high copy number in breast cancer lineages
  ** What is the mean ERB2B copy number in breast cancers?
* The gene dependency score should be < 0
  ** What is the mean gene dependency score in breast cancers?
''')


### Sanity checking with ERBB2 (2064)
Evaluating breast cancer lineages where at least one cell line had copy number > 2:
* Is ERB2B in the hotspot gene set?
* We expect ERBB2 to have high copy number in breast cancer lineages
  ** What is the mean ERB2B copy number in breast cancers?
* The gene dependency score should be < 0
  ** What is the mean gene dependency score in breast cancers?


In [113]:
report("ERBB2 is in hotspot gene set? <b>"+str(2064 in hotspot_genes).upper()+"</b>")
report('<pre>')

erb = cn.loc[2064]
nums = {}
is_high = set()
for c in cell2sublineage:
    lin = cell2sublineage.get(c)
    if lin is None or erb.get(c) is None:
        continue
        
    if 'breast' in lin:
        if nums.get(lin) is None:
            nums[lin] = []
        nums[lin].append(erb[c])
        if erb[c] >= 2:
            is_high.add(lin)
            #report(','.join([c,lin,str('%.2f'%erb[c])]))
for lin in nums:
    if lin in is_high:
        report("ERBB2 mean copy number for "+lin+" ("+str(len(nums[lin]))+" cell lines): "+str('%.2f'%np.mean(nums[lin])))
        
erb_gd = achilles.loc[2064]
nums = {}
for c in erb_gd.keys():
    lin = cell2sublineage.get(c)
    if lin is None or not 'breast' in lin:
        continue
        
    if nums.get(lin) is None:
        nums[lin] = []
    nums[lin].append(erb_gd[c])

for lin in nums:
    #if lin in is_high:
    report("ERBB2 mean gene dependency for "+lin+" ("+str(len(nums[lin]))+" cell lines): "+str('%.2f'%np.mean(nums[lin])))
        
report('</pre>')
    

ERBB2 is in hotspot gene set? <b>TRUE</b>

<pre>

ERBB2 mean copy number for breast_TNBC (27 cell lines): 1.89

ERBB2 mean copy number for breast_TPBC (5 cell lines): 9.59

ERBB2 mean copy number for breast_HER2Amp (11 cell lines): 14.84

ERBB2 mean gene dependency for breast_HER2Amp (6 cell lines): -0.83

ERBB2 mean gene dependency for breast_ERpos (7 cell lines): -0.27

ERBB2 mean gene dependency for breast_TNBC (15 cell lines): -0.28

</pre>

In [125]:
report('''
## Lineages with observed high copy number genes

* Go through the list of lineages with high copy number genes
* Plot gene dependency vs. copy number
* Highlight genes/cell-lines with copy number > 2 and gene dependency < 0 for specific lineages

### Copy number vs gene dependency
''')


## Lineages with observed high copy number genes

* Go through the list of lineages with high copy number genes
* Plot gene dependency vs. copy number
* Highlight genes/cell-lines with copy number > 2 and gene dependency < 0 for specific lineages

### Copy number vs gene dependency


In [129]:
for sublin in keep_lineage:
    cells = sublineage2cell[sublin]
    
    best = {}
    genes = set()
    for c in cells:
        try:
            cdf = cn[c]
            cdf = cdf[cdf >= 2]
            best[c] = set(cdf.index)
            for g in best[c]:
                genes.add(g)
                
        except Exception as e:
            #print('Error',e)
            continue
    
    if len(best) == 0:
        continue
    
    keep_cells = set()
    for c in ac_cells:
        if c in cells:
            keep_cells.add(c)
    if len(keep_cells) == 0:
        #print("no shared cells")
        continue

    keep_genes = set()
    for g in ac_genes:
        if g in genes:
            keep_genes.add(g)
    if len(keep_genes) == 0:
        print("no shared genes")
        continue
    
    try:
        ac_s = achilles[keep_cells]
        ac_s = ac_s.T[keep_genes].T
        cn_s = cn[keep_cells]
        cn_s = cn_s.T[keep_genes].T
    except:
        continue

    ac_data = ac_s.to_numpy().flatten()
    cn_data = cn_s.to_numpy().flatten()
    
    # remove NaN and Inf
    keep_ac = []
    keep_cn = []
    for i, d1 in enumerate(ac_data):
        d2 = cn_data[i]
        # Skip gene dependency > 0
        if d1 >= 0:
            continue
        # Skip copy number < 2
        if d2 < 2:
            continue
        
        if not np.isfinite(d1) and np.isfinite(d2):
            continue
        keep_ac.append(d1)
        keep_cn.append(d2)
    
    ac_data = keep_ac
    cn_data = keep_cn  
    
    plt.scatter(cn_orig,ac_orig,c='silver',label='other lineages')
    plt.scatter(cn_data,ac_data,c='r',label='this lineage: gene dep.<0; copy num >= 2')
    plt.legend()
    outfile = 'plots/'+sublin+'.png'
    plt.xlabel('copy number')
    plt.ylabel('Achilles gene dependency')
    plt.title(sublin)
    plt.savefig(outfile,dpi=100)
    plt.close()
    report('<img src="'+outfile+'" style="float:left">')

<img src="plots/eye_uveal_melanoma.png" style="float:left">

<img src="plots/soft_tissue_sarcoma_undifferentiated.png" style="float:left">

<img src="plots/breast_HER2Amp.png" style="float:left">

<img src="plots/bone_chordoma.png" style="float:left">

<img src="plots/skin_squamous.png" style="float:left">

<img src="plots/esophagus_adenocarcinoma.png" style="float:left">

<img src="plots/skin_epidermoid_carcinoma.png" style="float:left">

<img src="plots/upper_aerodigestive_buccal_mucosa.png" style="float:left">

<img src="plots/soft_tissue_fibrosarcoma.png" style="float:left">

<img src="plots/soft_tissue_epitheliod_sarcoma.png" style="float:left">

In [187]:
html = candidates.to_html(index=False).replace('<table','<table id="candidates"')
report(html)
report('<script></sc')

<table id="candidates" border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>DepMap_ID</th>
      <th>sublineage</th>
      <th>HUGO_symbol</th>
      <th>Entrez_ID</th>
      <th>copy_number</th>
      <th>achilles_gene_dep</th>
      <th>rnai_gene_dep</th>
      <th>hotspot</th>
      <th>rnaseq_expressed</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>ACH-001164</td>
      <td>soft_tissue_sarcoma_undifferentiated</td>
      <td>PDIA6</td>
      <td>10130</td>
      <td>3.738903</td>
      <td>-0.523579</td>
      <td>NaN</td>
      <td>True</td>
      <td>True</td>
    </tr>
    <tr>
      <td>ACH-001164</td>
      <td>soft_tissue_sarcoma_undifferentiated</td>
      <td>RIDA</td>
      <td>10247</td>
      <td>3.725001</td>
      <td>-0.440433</td>
      <td>NaN</td>
      <td>True</td>
      <td>True</td>
    </tr>
    <tr>
      <td>ACH-001164</td>
      <td>soft_tissue_sarcoma_undifferentiated</td>
      <td>EMC8</td>
      <td>10328</td>
  